# Web Scraping para la Recopilación de Datos de Aeropuertos en América

## Introducción:

Este Jupyter Notebook tiene como objetivo principal la recopilación de datos de aeropuertos en América mediante técnicas de web scraping. En el contexto de nuestro proyecto de base de datos orientada a grafos para la gestión de aeropuertos, es esencial contar con información actualizada y precisa sobre los aeropuertos en la región.

El web scraping nos permitirá extraer datos relevantes de fuentes en línea, como sitios web de autoridades de aviación y aerolíneas. Estos datos incluirán detalles sobre ubicaciones geográficas, nombres de aeropuertos, servicios disponibles, rutas operadas y otros atributos esenciales.

## Objetivos:

Obtener datos de aeropuertos en América a partir de fuentes en línea.
Extraer información clave, como nombres de aeropuertos, ubicaciones y detalles operativos.
Almacenar los datos recopilados en un formato adecuado para su posterior análisis y uso en nuestra base de datos orientada a grafos.

### Autor

Este Python Notebook está hecho por **Manrique Camacho P. (C01554)**

## Importación de librerias

* **Requests**: La biblioteca requests le permite al usuario realizar solicitudes HTTP a las páginas web que desee analizar, facilitando la descarga del contenido HTML de dichas páginas para su posterior procesamiento.

* **Beautiful Soup (bs4)**: Beautiful Soup es una herramienta útil para analizar y buscar elementos HTML en el contenido descargado. Permite al usuario buscar y extraer información específica de las páginas web, como títulos, párrafos, enlaces y más.

* **Selenium**: Cuando los sitios web utilizan JavaScript para cargar contenido dinámico, selenium se convierte en una opción valiosa. Con esta biblioteca, el usuario puede automatizar un navegador web para interactuar con el sitio web y extraer datos de páginas que requieren interacción.

* **Pandas**: Pandas es una biblioteca esencial para estructurar y manipular los datos extraídos. Permite al usuario crear DataFrames para organizar los datos en filas y columnas, lo que facilita las operaciones de limpieza, filtrado y procesamiento.

* **Matplotlib y Seaborn**: Estas bibliotecas son útiles para crear gráficos y visualizaciones si se desea presentar los datos recopilados de manera visual. Permiten personalizar gráficos para mostrar tendencias o patrones en los datos.

* **GeoPandas**: Si el proyecto involucra datos geoespaciales o la representación geográfica de aeropuertos, GeoPandas es una herramienta útil. Permite al usuario trabajar con datos geoespaciales, crear mapas y realizar análisis geoespaciales.

In [1]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import ui
import chromedriver_autoinstaller
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

## Web Scraping para Recopilación de Datos

En esta sección, se explorará el web scraping, una técnica automatizada para obtener datos de sitios web. Los temas clave incluyen:

* **Solicitudes HTTP**: Se utilizará la biblioteca requests para descargar el contenido HTML de las páginas web.
* **Análisis HTML**: Se empleará Selenium para interactuar con sitios web dinámicos y extraer información relevante.
* **Recopilación de Datos**: Se llevará a cabo la recolección de datos desde los sitios web seleccionados.

In [94]:
chromedriver_autoinstaller.install()  # Instala automáticamente la versión correcta de ChromeDriver
ae_centralamerica = ['pty/airport-panama-city-tocumen','sjo/airport-san-jose-juan-santamaria',
                    'sal/airport-san-salvador-cuscatlan','mga/airport-managua',
                    'tgu/airport-tegucigalpa'] 

In [10]:
res = requests.get("https://airportinfo.live/arrivals/sjo/airport-san-jose-juan-santamaria",headers ={'User-Agent': 'Mozilla/5.0'})

soup = bs4.BeautifulSoup(res.text, "html.parser")

hello


In [133]:
def airport_basic(extension): #Nos dara la información general del aeropuerto

    res = requests.get(f"https://airportinfo.live/arrivals/{extension}",headers ={'User-Agent': 'Mozilla/5.0'})
    
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    nombre = soup.find_all('strong')[-1].get_text(strip=True)
    telephone = ' '.join(soup.find_all('li')[-3].text.split()[2:])
    timezone = ' '.join(soup.find_all('li')[-2].text.split()[5:])

    data = []
    for i in soup.find_all('li')[-4]:
        data.append(i)
    
    location = data[2]
    
    info_ae = [nombre, location, timezone, telephone]
    
    return info_ae

In [171]:
driver = webdriver.Chrome()
driver.get("https://airportinfo.live/arrivals/sjo/airport-san-jose-juan-santamaria")
hora = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="timeStart"]')))
driver.execute_script("return arguments[0].scrollIntoView(true);", hora)

In [153]:
pais = driver.find_element(By.XPATH, '//*[@id="AV627"]/td[2]/a')
print(pais.text)

San Salvador [SAL]


In [172]:
body = driver.find_element(By.XPATH, '//*[@id="San José-arrivals"]/tbody')
print(body.text.split('\n'))

['AV 627', 'San Salvador [SAL]', 'Avianca 21:50', '[Oct-8]', '', '', '23:05', '23:00', 'TM', 'ARRIVED', 'AC 6137', 'San Salvador [SAL]', 'Air Canada 21:50', '[Oct-8]', '', '', '23:05', '23:00', 'TM', 'ARRIVED', 'Codeshare Flight - operated by AV627', 'IB 7803', 'San Salvador [SAL]', 'Iberia 21:50', '[Oct-8]', '', '', '23:05', '23:00', 'TM', 'ARRIVED', 'Codeshare Flight - operated by AV627', 'AV 651', 'Guatemala City [GUA]', 'Avianca 21:30', '[Oct-8]', '', '', '23:05', '22:56', 'TM', 'ARRIVED', 'AC 6122', 'Guatemala City [GUA]', '21:30', '[Oct-8]', '', '', '23:05', '22:56', 'TM', 'ARRIVED', 'Codeshare Flight - operated by AV651', 'AS 1390', 'Los Angeles [LAX]', 'Alaska Airlines 16:53', '17:40', '[Oct-8]', 'T664A', '23:44', '23:59', 'TMHG', 'IN AIR']


In [170]:
body.text.split()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x0000000100b4ad68 chromedriver + 4337000
1   chromedriver                        0x0000000100b42de4 chromedriver + 4304356
2   chromedriver                        0x000000010076fa5c chromedriver + 293468
3   chromedriver                        0x0000000100774cc8 chromedriver + 314568
4   chromedriver                        0x00000001007768dc chromedriver + 321756
5   chromedriver                        0x0000000100776a08 chromedriver + 322056
6   chromedriver                        0x00000001007af3d4 chromedriver + 553940
7   chromedriver                        0x00000001007aa200 chromedriver + 532992
8   chromedriver                        0x00000001007ef908 chromedriver + 817416
9   chromedriver                        0x00000001007a8a5c chromedriver + 526940
10  chromedriver                        0x0000